# SNN–PAC–Kuramoto Closed-Loop Demo

Full architecture sketch: spiking neural networks coupled to Kuramoto
oscillators through phase-amplitude coupling (PAC) gating, with
Lyapunov stability monitoring and DIRECTOR_AI guardrail integration.

**What this notebook covers:**

1. **LIF spiking layer** — minimal leaky integrate-and-fire population
2. **Rate → Ψ decoder** — spike rate maps to global driver phase
3. **Single-tick closed loop** — Kuramoto → LIF → rate decode → Ψ feedback
4. **100-tick trajectory** — 10 ms window @ 10 kHz with R convergence
5. **PAC cross-layer routing** — multi-layer SNN with PAC gating
6. **Lyapunov stability** — V(t) tracking with LyapunovGuard
7. **DIRECTOR_AI guardrail** — stability verdict export

References:
- [Paper 27 (Academia)](https://www.academia.edu/143833534/27_SCPN_The_Knm_Matrix)
- [arXiv:2004.06344](https://arxiv.org/abs/2004.06344) — Generalized Kuramoto–Sakaguchi
- [Paper 27 PDF](../docs/REVIEWER_PAPER27_INTEGRATION.pdf)

---

Copyright 1996–2026 Miroslav Šotek · GNU AGPL v3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scpn_control.phase.kuramoto import (
    kuramoto_sakaguchi_step, order_parameter, wrap_phase,
    lyapunov_v, lyapunov_exponent,
)
from scpn_control.phase.knm import build_knm_paper27, OMEGA_N_16
from scpn_control.phase.upde import UPDESystem
from scpn_control.phase.lyapunov_guard import LyapunovGuard

SEED = 42
rng = np.random.default_rng(SEED)
plt.rcParams.update({"figure.dpi": 120, "figure.facecolor": "white"})

## 1. Minimal LIF Spiking Layer

Leaky integrate-and-fire neuron population. Membrane dynamics:

$$\tau\,\frac{dV}{dt} = -(V - V_{\text{rest}}) + R_{\text{mem}}\,I_{\text{syn}}$$

Spike when $V \geq V_{\text{th}}$, then reset to $V_{\text{reset}}$.

In [ ]:
class LIFLayer:
    """Minimal LIF population."""
    __slots__ = ("n", "v", "v_rest", "v_th", "v_reset", "tau", "r_mem", "dt")

    def __init__(self, n: int, dt: float = 1e-3):
        self.n = n
        self.v = np.full(n, -65.0)
        self.v_rest = -65.0
        self.v_th = -55.0
        self.v_reset = -70.0
        self.tau = 10e-3
        self.r_mem = 1.0
        self.dt = dt

    def step(self, i_syn: np.ndarray) -> np.ndarray:
        dv = (-(self.v - self.v_rest) + self.r_mem * i_syn) / self.tau
        self.v += self.dt * dv
        spikes = self.v >= self.v_th
        self.v[spikes] = self.v_reset
        return spikes


def rate_to_psi(spikes: np.ndarray, nu_max: float = 100.0) -> float:
    """Decode spike rate to global phase Ψ ∈ (−π, π]."""
    nu = float(np.mean(spikes)) / 1e-3  # spikes/s (1 ms window)
    return np.pi * (2.0 * min(nu / nu_max, 1.0) - 1.0)


# Verify LIF fires with strong input
lif_test = LIFLayer(64)
spikes_test = lif_test.step(np.full(64, 15.0))
print(f"LIF test: {int(spikes_test.sum())}/64 neurons spiked")
print(f"Rate→Ψ: {rate_to_psi(spikes_test):.3f} rad")

## 2. Single-Tick Closed Loop

One full tick of the SNN–Kuramoto loop:

1. Kuramoto step with current Ψ → R, ψ_r
2. Inject R·cos(ψ_r − preferred) into LIF synaptic current
3. LIF integration → spike/no-spike
4. Decode spike rate → Ψ_next for feedback

In [ ]:
N_osc = 1000
N_lif = 64
theta = rng.uniform(-np.pi, np.pi, N_osc)
omega = rng.normal(0, 0.5, N_osc)
lif = LIFLayer(N_lif)
psi = 0.0

# One closed-loop tick
out = kuramoto_sakaguchi_step(
    theta, omega, dt=1e-3, K=2.0, zeta=0.5,
    psi_driver=psi, psi_mode="external",
)
theta = out["theta1"]
R = out["R"]
psi_r = out["Psi_r"]

# Kuramoto coherence → LIF synaptic current
preferred = np.linspace(0, 2 * np.pi, N_lif, endpoint=False)
i_syn = 10.0 + 5.0 * R * np.cos(psi_r - preferred)
spikes = lif.step(i_syn)

# Rate decode → Ψ feedback
psi_next = rate_to_psi(spikes)

print(f"R = {R:.4f}, ψ_r = {psi_r:.4f}")
print(f"Spikes: {int(spikes.sum())}/{N_lif}")
print(f"Ψ_next = {psi_next:.4f} rad")

## 3. 100-Tick Closed-Loop Trajectory (10 ms @ 10 kHz)

Run the full loop for 100 ticks and track R(t), Ψ(t), spike rate,
and Lyapunov V(t) = (1/N) Σ (1 − cos(θ_i − Ψ)).

In [ ]:
N_osc = 1000
N_lif = 64
n_ticks = 500
dt_tick = 1e-3

theta = rng.uniform(-np.pi, np.pi, N_osc)
omega = rng.normal(0, 0.5, N_osc)
lif = LIFLayer(N_lif, dt=dt_tick)
psi = 0.0

R_hist = np.empty(n_ticks)
psi_hist = np.empty(n_ticks)
rate_hist = np.empty(n_ticks)
V_hist = np.empty(n_ticks)

preferred = np.linspace(0, 2 * np.pi, N_lif, endpoint=False)

for t in range(n_ticks):
    out = kuramoto_sakaguchi_step(
        theta, omega, dt=dt_tick, K=2.0, zeta=1.5,
        psi_driver=psi, psi_mode="external",
    )
    theta = out["theta1"]
    R_hist[t] = out["R"]

    i_syn = 10.0 + 5.0 * out["R"] * np.cos(out["Psi_r"] - preferred)
    spikes = lif.step(i_syn)
    psi = rate_to_psi(spikes)
    psi_hist[t] = psi
    rate_hist[t] = float(np.mean(spikes)) / dt_tick
    V_hist[t] = lyapunov_v(theta, psi)

t_ms = np.arange(n_ticks) * dt_tick * 1000

fig, axes = plt.subplots(4, 1, figsize=(11, 10), sharex=True)

axes[0].plot(t_ms, R_hist, color="navy", lw=1.5)
axes[0].axhline(0.95, color="red", ls="--", lw=0.8)
axes[0].set_ylabel("R(t)")
axes[0].set_title("SNN–Kuramoto Closed Loop — 500 ticks")

axes[1].plot(t_ms, psi_hist, color="darkorange", lw=1)
axes[1].set_ylabel("Ψ(t) [rad]")
axes[1].set_ylim(-np.pi, np.pi)

axes[2].plot(t_ms, rate_hist, color="green", lw=1)
axes[2].set_ylabel("Spike rate [Hz]")

axes[3].plot(t_ms, V_hist, color="purple", lw=1.5)
axes[3].set_ylabel("V(t) Lyapunov")
axes[3].set_xlabel("Time (ms)")

plt.tight_layout()
plt.show()

lam = lyapunov_exponent(V_hist.tolist(), dt_tick)
print(f"R: {R_hist[0]:.3f} → {R_hist[-1]:.3f}")
print(f"V: {V_hist[0]:.4f} → {V_hist[-1]:.4f}")
print(f"λ = {lam:.4f} ({'STABLE' if lam < 0 else 'UNSTABLE'})")

## 4. PAC Cross-Layer Routing

Multi-layer SNN with PAC gating. Each SCPN layer has its own
LIF population. The PAC gate modulates inter-layer weights:

$$g_{n \to m} = 1 + \gamma_{\text{PAC}} \cdot (1 - R_n)$$

When a source layer is incoherent (low R), the gate amplifies
coupling — desynchronised layers drive downstream amplitude modulation.

Architecture:
```
L1 (ω=1.33) ──K[0,6]──► L7 (ω=1.06)
   LIF₁ (64)     PAC g₁₇     LIF₇ (64)
       │                         │
       └────── Kuramoto Bus ─────┘
              R_m, ψ_m per layer
              PAC → SNN weight mod
              Spike rate → Ψ
```

In [ ]:
L = 4
N_per = 60
N_lif_per = 32
n_steps = 1000
dt = 0.005
R_target = 0.85
Kp_snn = 2.5

spec = build_knm_paper27(L=L, zeta_uniform=1.0)
upde = UPDESystem(spec=spec, dt=dt, psi_mode="external")

th = [rng.uniform(-np.pi, np.pi, N_per) for _ in range(L)]
om = [OMEGA_N_16[m] + rng.normal(0, 0.2, N_per) for m in range(L)]
lifs = [LIFLayer(N_lif_per, dt=dt) for _ in range(L)]
psi_layers = np.zeros(L)

R_hist = np.empty((n_steps, L))
R_global_hist = np.empty(n_steps)
gain_hist = np.empty((n_steps, L))
V_hist = np.empty(n_steps)

for t in range(n_steps):
    # Global Ψ from mean of per-layer Ψ
    psi_global = float(np.mean(psi_layers))
    out = upde.step(th, om, psi_driver=psi_global, pac_gamma=1.5)
    th = out["theta1"]
    R_m = out["R_layer"]
    psi_m = out["Psi_layer"]
    R_hist[t] = R_m
    R_global_hist[t] = out["R_global"]
    V_hist[t] = out["V_global"]

    for m in range(L):
        # SNN error-driven gain
        error = R_target - R_m[m]
        gain = max(0.2, 1.0 + Kp_snn * error)
        gain_hist[t, m] = gain

        # Inject Kuramoto coherence into LIF
        pref = np.linspace(0, 2 * np.pi, N_lif_per, endpoint=False)
        i_syn = 10.0 * gain + 5.0 * R_m[m] * np.cos(psi_m[m] - pref)
        spikes = lifs[m].step(i_syn)
        psi_layers[m] = rate_to_psi(spikes)

t_s = np.arange(n_steps) * dt

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

for m in range(L):
    axes[0].plot(t_s, R_hist[:, m], lw=1.2, label=f"L{m+1}")
axes[0].axhline(R_target, color="red", ls="--", lw=0.8, label="R_target")
axes[0].set_ylabel("R_m(t)")
axes[0].set_title("PAC Cross-Layer SNN — Per-Layer Coherence")
axes[0].legend(fontsize=8, ncol=5)
axes[0].set_ylim(0, 1.05)

for m in range(L):
    axes[1].plot(t_s, gain_hist[:, m], lw=1, label=f"L{m+1}")
axes[1].set_ylabel("SNN gain")
axes[1].set_title("SNN Proportional Controller Gains")
axes[1].legend(fontsize=8, ncol=5)

axes[2].plot(t_s, V_hist, color="purple", lw=1.5)
axes[2].set_ylabel("V(t) Lyapunov")
axes[2].set_xlabel("Time (s)")

plt.tight_layout()
plt.show()

lam = lyapunov_exponent(V_hist.tolist(), dt)
print(f"R_global: {R_global_hist[0]:.3f} → {R_global_hist[-1]:.3f}")
print(f"λ = {lam:.4f} ({'STABLE' if lam < 0 else 'UNSTABLE'})")

## 5. LyapunovGuard — Real-Time Stability Monitor

The `LyapunovGuard` monitors V(t) over a sliding window and flags
instability when λ > 0 for K consecutive windows.

Interface mirrors DIRECTOR_AI's `CoherenceScorer` → `CoherenceScore`.

In [ ]:
guard = LyapunovGuard(window=50, dt=1e-3, max_violations=3)

# Stable trajectory: ζ pulls toward Ψ
theta_g = rng.uniform(-np.pi, np.pi, 200)
omega_g = np.zeros(200)
psi_g = 0.5

verdicts = []
for t in range(200):
    dth = 3.0 * np.sin(psi_g - theta_g)
    theta_g = wrap_phase(theta_g + 0.01 * dth)
    v = guard.check(theta_g, psi_g)
    verdicts.append(v)

scores = [v.score for v in verdicts]
lambdas = [v.lambda_exp for v in verdicts]

fig, axes = plt.subplots(2, 1, figsize=(10, 5), sharex=True)

axes[0].plot(scores, color="green", lw=1.5)
axes[0].set_ylabel("Stability score")
axes[0].set_title("LyapunovGuard — Online Monitoring")
axes[0].set_ylim(0, 1.05)

axes[1].plot(lambdas, color="navy", lw=1.5)
axes[1].axhline(0, color="red", ls="--", lw=0.8)
axes[1].set_ylabel("λ (Lyapunov exp)")
axes[1].set_xlabel("Step")

plt.tight_layout()
plt.show()

final = verdicts[-1]
print(f"Final verdict: approved={final.approved}, λ={final.lambda_exp:.4f}, score={final.score:.4f}")

## 6. DIRECTOR_AI Guardrail Export

Export the `LyapunovVerdict` in DIRECTOR_AI `AuditLogger` format
for integration with the dual-entropy coherence framework.

In [ ]:
import json

# Batch check from the UPDE trajectory
guard_batch = LyapunovGuard(dt=dt)
verdict_batch = guard_batch.check_trajectory(V_hist.tolist())

print(f"Batch verdict: approved={verdict_batch.approved}, λ={verdict_batch.lambda_exp:.4f}")
print()

# Export to DIRECTOR_AI format
audit_entry = guard_batch.to_director_ai_dict(verdict_batch)
print("DIRECTOR_AI AuditLogger entry:")
print(json.dumps(audit_entry, indent=2))

## Summary

| Component | Status |
|-----------|--------|
| LIF spiking layer | 64 neurons, 1 ms timestep |
| Rate → Ψ decoder | ν/ν_max → (−π, π] |
| Single-tick closed loop | Kuramoto → LIF → Ψ feedback |
| 500-tick trajectory | R convergence + Lyapunov V decay |
| PAC cross-layer (4 layers) | PAC γ=1.5, per-layer SNN gains |
| LyapunovGuard | Sliding window λ monitor |
| DIRECTOR_AI export | AuditLogger-compatible dict |

All components verified with 44 Python + 9 Rust tests.